In [1]:
# -*- coding: utf-8 -*-
file_path = "./datas/hashcode_classification2020_train.csv"
file_path2 = "./datas/hashcode_classification2020_test.csv"
import re
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
import Bi_LSTM as Bi_LSTM
import numpy as np
from konlpy.tag import Okt
import json
import os
from pprint import pprint
import nltk
import keras
from gensim.models import Word2Vec
import pandas as pd

Using TensorFlow backend.


In [2]:
def read_data(file_path):
    datas = pd.read_csv(file_path, engine='python', names=['title', 'content', 'label'], encoding='utf-8')
    title = datas.loc[1:,'title'].values
    content = datas.loc[1:,'content'].values
    label = datas.loc[1:,'label'].values
    result_data = []
    for idx, title_data in enumerate(title):
        tmp_data = [title_data, content[idx], label[idx]]
        result_data.append(tmp_data)
    return result_data
train_data = read_data(file_path)
test_data = read_data(file_path2)

In [3]:
def tokenize(doc):
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    okt = Okt()
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

In [4]:
def modeling(x_train, y_train):
    model = models.Sequential()
    model.add(layers.Dense(512, input_shape=(6053, ), activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    model.compile(optimizer=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8),
                  loss=losses.sparse_categorical_crossentropy,
                  metrics=[metrics.sparse_categorical_accuracy])
    tb_hist = keras.callbacks.TensorBoard(log_dir='./graphs', histogram_freq=0, write_graph=True,
                                          write_images=True, profile_batch = 100000000)
    model.fit(x_train, y_train, epochs=25, batch_size=512, callbacks=[tb_hist])
    return model

In [5]:
def array_to_float(data):
    x_train = np.asarray(data).astype('float32')
    return x_train
def array_to_int(data):
    x_train = np.asarray(data).astype('int32')
    return x_train

In [6]:
def change_frequency(docs):
    train_x = [term_frequency(d) for d, _ in docs]
    train_y = [c for _, c in docs]
    return train_x, train_y

In [7]:
def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

In [8]:
def make_selected_words(train_docs):
    tokens = [t for d in train_docs for t in d[0]]
    text = nltk.Text(tokens, name='NMSC')
    selected_words = [f[0] for f in text.vocab().most_common(10000)]
    return selected_words

In [9]:
def read_and_tokenize():
    if os.path.isfile('train_docs.json'):
        with open('train_docs.json', encoding="utf-8") as f:
            train_docs = json.load(f)
    else:
        hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
        train_docs = [(tokenize(hangul.sub("", str(row[0]))),row[1], row[2]) for row in train_data]
        # JSON 파일로 저장
        with open('train_docs.json', 'w', encoding="utf-8") as make_file:
            json.dump(train_docs, make_file, ensure_ascii=False, indent="\t")
    return train_docs

In [10]:
def read_and_tokenize2():
    if os.path.isfile('test_docs.json'):
        with open('test_docs.json', encoding="utf-8") as f:
            test_docs = json.load(f)
    else:
        hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
        test_docs = [(tokenize(hangul.sub("", str(row[0]))),row[1], row[2]) for row in test_data]
        # JSON 파일로 저장
        with open('train_docs.json', 'w', encoding="utf-8") as make_file:
            json.dump(test_docs, make_file, ensure_ascii=False, indent="\t")
    return test_docs

In [11]:
train_docs = read_and_tokenize()

In [12]:
test_docs = read_and_tokenize2()

JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.

In [13]:
selected_words = make_selected_words(train_docs)

In [14]:
train_x, train_y = change_frequency(train_docs)


In [15]:
print(len(train_x[3]))

6053


In [16]:
model = modeling(array_to_float(train_x),array_to_float(train_y))

Train on 2592 samples
Epoch 1/25
2592/2592 [==============================] - 1s 209us/sample - loss: 2.0983 - sparse_categorical_accuracy: 0.2504
Epoch 2/25
2592/2592 [==============================] - 0s 100us/sample - loss: 1.5941 - sparse_categorical_accuracy: 0.5876
Epoch 3/25
2592/2592 [==============================] - 0s 116us/sample - loss: 1.1819 - sparse_categorical_accuracy: 0.6852
Epoch 4/25
2592/2592 [==============================] - 0s 108us/sample - loss: 0.8927 - sparse_categorical_accuracy: 0.7535
Epoch 5/25
2592/2592 [==============================] - 0s 127us/sample - loss: 0.6942 - sparse_categorical_accuracy: 0.7890
Epoch 6/25
2592/2592 [==============================] - 0s 111us/sample - loss: 0.5239 - sparse_categorical_accuracy: 0.8422
Epoch 7/25
2592/2592 [==============================] - 0s 116us/sample - loss: 0.4012 - sparse_categorical_accuracy: 0.8962
Epoch 8/25
2592/2592 [==============================] - 0s 124us/sample - loss: 0.3113 - sparse_categor

In [17]:
def predict_pos_neg(review):
    token = tokenize(review)
    tf = term_frequency(token)
    data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
    print(model.predict(data))

In [19]:
predict_pos_neg(test_data[0][1])

JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.